In [3]:
from pydataxm import * #Requiere instalación: pip install pydataxm
import datetime as dt   
import pandas as pd

class ConsultaSinergox():
    """ 
    Crear conexión con la API xm, definir la fecha inicial y fecha final de consulta.
    método TraerDatos requiere los argumentos de Métrica y entidad (Entidad se refiere
    al tipo de agrupación, si indivudual por recurso (Agente) o total del sistema (Sistema))

    """
    def __init__(self,sFechaIni,sFechaFin):
        self.objetoAPI = pydataxm.ReadDB() 
        self.sFormatoFecha='%Y-%m-%d'
        self.dfEstructura =self.objetoAPI.get_collections() #El método get_collection sin argumentos retorna todas las variables que se tienen disponible en la API y que se pueden consultar  
        self.dtFechaIni=dt.datetime.strptime(sFechaIni, self.sFormatoFecha).date()
        self.dtFechaFin=dt.datetime.strptime(sFechaFin, self.sFormatoFecha).date()

        
    def TraerDatos(self,sMetrica,sEntidad):
        dfRecursos=self.objetoAPI.request_data(sMetrica,
                                           sEntidad,
                                            self.dtFechaIni,
                                            self.dtFechaFin)
        return(dfRecursos)

    def CrearSerieTiempo(self, recDF, genDF):
        """Función que retorna dataframe de datos de genreación Real por recurso en serie de tiempo"""
        
        #Union de tablas de generación y recursos.
        merge_df = recDF.merge(genDF, left_on='Values_Code', right_on='Values_code').fillna(0)
        columnas_usar = ['Date_y','Values_Code', 'Values_Name', 'Values_Type', 'Values_Disp',
        'Values_RecType', 'Values_CompanyCode', 'Values_EnerSource',
        'Values_OperStartdate', 'Values_State']
        columnas_valores =['Values_Hour01', 'Values_Hour02', 'Values_Hour03', 'Values_Hour04',
        'Values_Hour05', 'Values_Hour06', 'Values_Hour07', 'Values_Hour08',
        'Values_Hour09', 'Values_Hour10', 'Values_Hour11', 'Values_Hour12',
        'Values_Hour13', 'Values_Hour14', 'Values_Hour15', 'Values_Hour16',
        'Values_Hour17', 'Values_Hour18', 'Values_Hour19', 'Values_Hour20',
        'Values_Hour21', 'Values_Hour22', 'Values_Hour23', 'Values_Hour24']
        #Tabla pivoteada con datos de generación por tipo de recurso
        df_gen_series = pd.melt(merge_df, id_vars=columnas_usar, value_vars= columnas_valores, value_name = 'kWh', var_name= 'Hora')
        #Ajuste de la hora para convertirlo en serie de tiempo
        df_gen_series['Hora'] = [x[-2:] for x in list(df_gen_series.Hora)] #Retiro del nombre Value_Hour
        df_gen_series['Hora'] = df_gen_series['Hora'].str.replace('24','00') #Cambio de periodo 24 por Hora 00
        df_gen_series['Fecha'] = df_gen_series['Hora'].apply(lambda _: _ + ':00:00') #Concatenación string para formato HH:MM:SS
        df_gen_series['Fecha'] = df_gen_series['Date_y'].dt.strftime('%Y-%m-%d') + " " + df_gen_series['Fecha'] #Concatenación Fecha con hora
        df_gen_series['Fecha'] = df_gen_series['Fecha'].apply(lambda _: dt.datetime.strptime(_,'%Y-%m-%d %H:%M:%S')) #Conversión en tipo datetime
        df_gen_series.index = df_gen_series.Fecha #Se asigna como indice la columna fecha tipo datetime
        df_gen_series = df_gen_series.drop(columns=['Date_y','Hora','Fecha']) #se retiran columnas utilizadas
        df_gen_series.sort_index() #Se ordena por indice en orden ascendente
        
        return df_gen_series


In [4]:
claseAPI=ConsultaSinergox('2021-01-01','2021-12-31')
#Listado de recursos o plantas del sistema
df_recursos_sistema = claseAPI.TraerDatos('ListadoRecursos', 'Sistema')
#Datos de generación horaria por recurso
df_gen_Recurso = claseAPI.TraerDatos('Gene', 'Recurso')

df_gen_serietiempo = claseAPI.CrearSerieTiempo(df_recursos_sistema,df_gen_Recurso)
df_gen_serietiempo

,Values_Code,Values_Name,Values_Type,Values_Disp,Values_RecType,Values_CompanyCode,Values_EnerSource,Values_OperStartdate,Values_State,kWh
Fecha,,,,,,,,,,
2021-01-01 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,19932.61
2021-01-02 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,17596.15
2021-01-03 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,19903.55
2021-01-04 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,19688.98
2021-01-05 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,15640.11
...,...,...,...,...,...,...,...,...,...,...
2021-12-26 00:00:00,ZPA5,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,60861.12
2021-12-27 00:00:00,ZPA5,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,40141.44
2021-12-28 00:00:00,ZPA5,ZIPAEMG 5,TERMICA,DESPACHADO CENTRALMENTE,NORMAL,ENDG,CARBON,1985-12-14,OPERACION,44901.33


In [10]:
df_gen_serie_filo = df_gen_serietiempo[df_gen_serietiempo.Values_RecType.str.contains('FILO')]
df_gen_serie_filo

,Values_Code,Values_Name,Values_Type,Values_Disp,Values_RecType,Values_CompanyCode,Values_EnerSource,Values_OperStartdate,Values_State,kWh
Fecha,,,,,,,,,,
2021-01-01 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,19932.61
2021-01-02 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,17596.15
2021-01-03 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,19903.55
2021-01-04 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,19688.98
2021-01-05 01:00:00,2QBW,EL POPAL,HIDRAULICA,NO DESPACHADO CENTRALMENTE,FILO DE AGUA,ISGG,AGUA,2014-03-31,OPERACION,15640.11
...,...,...,...,...,...,...,...,...,...,...
2021-12-27 00:00:00,SNFR,SAN FRANCISCO,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,EPMG,AGUA,1969-01-01,OPERACION,49624.19
2021-12-28 00:00:00,SNFR,SAN FRANCISCO,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,EPMG,AGUA,1969-01-01,OPERACION,76164.49
2021-12-29 00:00:00,SNFR,SAN FRANCISCO,HIDRAULICA,DESPACHADO CENTRALMENTE,FILO DE AGUA,EPMG,AGUA,1969-01-01,OPERACION,18101.08


In [14]:
df_gen_serie_filo = df_gen_serie_filo.groupby('Fecha').sum()

In [15]:
df_gen_serie_filo.to_csv('DataGeneración_filo.csv', index = True) 